In [ ]:
%pip install --upgrade --quiet langchain-core

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.


In [2]:

# from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import getpass
import os

# if not os.environ.get("GEMINI_API_KEY"):
#   os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

google_api_key = os.getenv("GEMINI_API_KEY")
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY not found in environment variables. Please set it in a .env file.")
# --- END CHANGE ---

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key, temperature=0.0)

# llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:


tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='Positive', aggressiveness=1, language='Spanish')

### Spam detection   

In [ ]:
#  Define the Prompt Template (remains the same)
prompt_template = PromptTemplate(
    input_variables=["tweet_content"],
    template="""You are an expert tweet spam detector. Your task is to analyze the following tweet and determine if it is SPAM or NOT_SPAM.
A tweet is considered SPAM if it contains:
- Excessive promotional content or unsolicited advertising.
- Phishing links or attempts to steal personal information.
- Impersonation or misleading claims.
- Repetitive, generic messages sent to many users.
- Requests for money or personal details in a suspicious manner.

Classify the tweet as either 'SPAM' or 'NOT_SPAM'. Do not include any other text or explanation.

Tweet: "{tweet_content}"
Classification:"""
)

spam_detection_chain = LLMChain(llm=llm, prompt=prompt_template)

print("--- Running Spam Detection Examples ---")

# Test 1: Likely Spam
tweet_1 = "💰 Earn $5000 in 2 days! Click this link now: bit.ly/freespins2025 #makemoney #crypto"
print(f"\nTweet: \"{tweet_1}\"")
prediction_1 = spam_detection_chain.run(tweet_1)
print(f"Prediction: {prediction_1.strip()}")

# Test 2: Not Spam
tweet_2 = "Just finished my morning coffee and starting to code! What are you working on today? #coding #developer"
print(f"\nTweet: \"{tweet_2}\"")
prediction_2 = spam_detection_chain.run(tweet_2)
print(f"Prediction: {prediction_2.strip()}")

# Test 3: Another Spam example
tweet_3 = "Congratulations! You've won a free iPhone! DM me your address to claim. Limited time offer! #giveaway"
print(f"\nTweet: \"{tweet_3}\"")
prediction_3 = spam_detection_chain.run(tweet_3)
print(f"Prediction: {prediction_3.strip()}")

# Test 4: Edge case / potentially borderline
tweet_4 = "Check out my new blog post on LangChain! It's super helpful: myblog.com/langchain-guide #AI #LangChain"
print(f"\nTweet: \"{tweet_4}\"")
prediction_4 = spam_detection_chain.run(tweet_4)
print(f"Prediction: {prediction_4.strip()}")

C:\Users\Ai_Explore\AppData\Local\Temp\ipykernel_5264\550997978.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  spam_detection_chain = LLMChain(llm=llm, prompt=prompt_template)
C:\Users\Ai_Explore\AppData\Local\Temp\ipykernel_5264\550997978.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  prediction_1 = spam_detection_chain.run(tweet_1)


--- Running Spam Detection Examples ---

Tweet: "💰 Earn $5000 in 2 days! Click this link now: bit.ly/freespins2025 #makemoney #crypto"
Prediction: SPAM

Tweet: "Just finished my morning coffee and starting to code! What are you working on today? #coding #developer"
Prediction: NOT_SPAM

Tweet: "Congratulations! You've won a free iPhone! DM me your address to claim. Limited time offer! #giveaway"
Prediction: SPAM

Tweet: "Check out my new blog post on LangChain! It's super helpful: myblog.com/langchain-guide #AI #LangChain"
Prediction: NOT_SPAM
